In [2]:
import sys, os

# Chemin vers le dossier src de ton projet
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../music-recommender"))
src_path = os.path.join(project_root, "src")

# Ajoute le chemin au PYTHONPATH si pas déjà présent
if src_path not in sys.path:
    sys.path.append(src_path)

print("✅ Path ajouté :", src_path)

# Import des fonctions
from vectorization.embedding_cleaner import (
    load_dataset, clean_text, prepare_texts,
    generate_embeddings, save_with_embeddings
)

print("✅ Import réussi !")


✅ Path ajouté : c:\Users\artmu\Desktop\Arthur\Etude\Etude_sup\IUT\Cours\3A\S5\IA\music-recommender\src
✅ Import réussi !


In [3]:
df = load_dataset("../data/processed/sample_albums.csv")
texts = prepare_texts(df)
embeddings = generate_embeddings(texts)
save_with_embeddings(df, embeddings, "../data/processed/sample_albums_embedded.parquet")


✅ 23994 lignes chargées depuis ../data/processed/sample_albums.csv
🧠 Chargement du modèle all-MiniLM-L6-v2 ...


Batches: 100%|██████████| 750/750 [29:56<00:00,  2.40s/it]


✅ Embeddings générés.
✅ Données sauvegardées dans ../data/processed/sample_albums_embedded.parquet
